In [1]:
import torch
from torch_geometric.data import Data
from graphein.protein.config import ProteinGraphConfig
from graphein.protein.graphs import construct_graph
from graphein.protein.edges.distance import add_distance_threshold
from functools import partial
import esm
import networkx as nx

# Load ESM-2 model
protein_model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
protein_model.eval()

# Configure Graphein
config = ProteinGraphConfig(
    edge_construction_functions=[
        partial(add_distance_threshold, long_interaction_threshold=0, threshold=8)
    ]
)

def get_esm2_embedding(seq, pdb_id):
    # Handle sequences longer than 1022 residues
    if len(seq) > 1022:
        seq_feat = []
        for i in range(0, len(seq), 1022):
            data = [(pdb_id, seq[i:i+1022])]
            batch_labels, batch_strs, batch_tokens = batch_converter(data)
            with torch.no_grad():
                results = protein_model(batch_tokens, repr_layers=[33], return_contacts=True)
            token_representations = results["representations"][33]
            seq_feat.append(token_representations.squeeze(0)[1:len(data[0][1])+1])
        seq_feat = torch.cat(seq_feat, dim=0)
    else:
        data = [(pdb_id, seq)]
        batch_labels, batch_strs, batch_tokens = batch_converter(data)
        with torch.no_grad():
            results = protein_model(batch_tokens, repr_layers=[33], return_contacts=True)
        token_representations = results["representations"][33]
        seq_feat = token_representations.squeeze(0)[1:len(data[0][1])+1]
    
    return seq_feat

def adj2edge_index(adj):
    # Convert adjacency matrix to edge index format
    edge_index = adj.nonzero().t().contiguous()
    return edge_index

def protein_graph(pdb_path, pdb_id):
    # Construct graph using Graphein
    g = construct_graph(config=config, pdb_path=f"{pdb_path}")
    
    # Extract sequence
    seq = ''.join(g.graph[key] for key in g.graph.keys() if key.startswith("sequence_"))
    
    # Get node features using ESM-2
    node_feat = get_esm2_embedding(seq, pdb_id)
    
    # Build edges
    A = nx.to_numpy_array(g, nonedge=0, weight='distance')
    edge_index = adj2edge_index(torch.tensor(A))
    
    # Add extra structural information
    extra_features = torch.tensor([
        [g.nodes[i].get('is_helix', 0),
         g.nodes[i].get('is_sheet', 0),
         g.nodes[i].get('is_turn', 0),
         g.nodes[i].get('solvent_accessibility', 0)] 
        for i in range(len(seq))
    ])
    
    node_feat = torch.cat([node_feat, extra_features], dim=1)
    
    # Create Data object
    data = Data(x=node_feat, edge_index=edge_index, pos=torch.tensor([g.nodes[i]['coords'] for i in range(len(seq))]))
    
    return data

# Usage example
if __name__ == "__main__":
    pdb_path = "./../Testset/1a1e/1a1e_protein.pdb"
    pdb_id = "1a1e"  # Example PDB ID
    
    protein_graph_data = protein_graph(pdb_path, pdb_id)
    
    print(f"Protein {pdb_id}:")
    print(f"Node features: {protein_graph_data.x.shape}")
    print(f"Edge index: {protein_graph_data.edge_index.shape}")
    print(f"3D coordinates: {protein_graph_data.pos.shape}")

[09/08/24 22:02:09] WARNING  To use the Graphein submodule                                         ]8;id=275180;file:///Users/megagatlingpea/miniforge3/envs/torch/lib/python3.11/site-packages/graphein/protein/features/sequence/embeddings.py\embeddings.py]8;;\:]8;id=490177;file:///Users/megagatlingpea/miniforge3/envs/torch/lib/python3.11/site-packages/graphein/protein/features/sequence/embeddings.py#45\45]8;;\
                             graphein.protein.features.sequence.embeddings, you need to install:                   
                             biovec                                                                                
                             biovec cannot be installed via conda                                                  
                             Alternatively, you can install graphein with the extras:                              
                                                                                                                   
                             pip install graphein[extras]                                                          

                    WARNING  To use the Graphein submodule graphein.protein.visualisation, you  ]8;id=234218;file:///Users/megagatlingpea/miniforge3/envs/torch/lib/python3.11/site-packages/graphein/protein/visualisation.py\visualisation.py]8;;\:]8;id=188480;file:///Users/megagatlingpea/miniforge3/envs/torch/lib/python3.11/site-packages/graphein/protein/visualisation.py#36\36]8;;\
                             need to install: pytorch3d                                                            
                             To do so, use the following command: conda install -c pytorch3d                       
                             pytorch3d                                                                             

                    WARNING  To use the Graphein submodule graphein.protein.meshes, you need to        ]8;id=296451;file:///Users/megagatlingpea/miniforge3/envs/torch/lib/python3.11/site-packages/graphein/protein/meshes.py\meshes.py]8;;\:]8;id=783704;file:///Users/megagatlingpea/miniforge3/envs/torch/lib/python3.11/site-packages/graphein/protein/meshes.py#30\30]8;;\
                             install: pytorch3d                                                                    
                             To do so, use the following command: conda install -c pytorch3d pytorch3d             

AttributeError: module 'esm' has no attribute 'pretrained'